In [40]:
from abc import ABC, abstractmethod
from enum import Enum
import enum
import sqlite3
import threading
from typing import Optional, Type, TypeVar, Union
from PIL.Image import Image as PILImage
from pydantic import BaseModel, Field
from torch import Tensor

In [41]:

class ResourceOrigin(str, Enum):
    """The origin of a resource (eg image or tensor)."""

    RESULTS = "results"
    UPLOADS = "uploads"
    INTERMEDIATES = "intermediates"


class ImageKind(str, Enum):
    """The kind of an image. Use ImageKind.OTHER for non-default kinds."""

    IMAGE = "image"
    CONTROL_IMAGE = "control_image"
    OTHER = "other"


class TensorKind(str, Enum):
    """The kind of a tensor. Use TensorKind.OTHER for non-default kinds."""

    IMAGE_LATENTS = "image_latents"
    CONDITIONING = "conditioning"
    OTHER = "other"


In [42]:

def create_sql_values_string_from_string_enum(enum: Type[Enum]):
    """
    Creates a string of the form "('value1'), ('value2'), ..., ('valueN')" from a StrEnum.
    """

    delimiter = ", "
    values = [f"('{e.value}')" for e in enum]
    return delimiter.join(values)


def create_sql_table_from_enum(
    enum: Type[Enum],
    table_name: str,
    primary_key_name: str,
    conn: sqlite3.Connection,
    cursor: sqlite3.Cursor,
    lock: threading.Lock,
):
    """
    Creates and populates a table to be used as a functional enum.
    """

    try:
        lock.acquire()

        values_string = create_sql_values_string_from_string_enum(enum)

        cursor.execute(
            f"""--sql
            CREATE TABLE IF NOT EXISTS {table_name} (
                {primary_key_name} TEXT PRIMARY KEY
            );
            """
        )
        cursor.execute(
            f"""--sql
            INSERT OR IGNORE INTO {table_name} ({primary_key_name}) VALUES {values_string};
            """
        )
        conn.commit()
    finally:
        lock.release()


"""
`resource_origins` functions as an enum for the ResourceOrigin model.
"""


# def create_resource_origins_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
#     create_sql_table_from_enum(
#         enum=ResourceOrigin,
#         table_name="resource_origins",
#         primary_key_name="origin_name",
#         conn=conn,
#         cursor=cursor,
#         lock=lock,
#     )


"""
`image_kinds` functions as an enum for the ImageType model.
"""


# def create_image_kinds_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    # create_sql_table_from_enum(
    #     enum=ImageKind,
    #     table_name="image_kinds",
    #     primary_key_name="kind_name",
    #     conn=conn,
    #     cursor=cursor,
    #     lock=lock,
    # )


"""
`tensor_kinds` functions as an enum for the TensorType model.
"""


# def create_tensor_kinds_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    # create_sql_table_from_enum(
    #     enum=TensorKind,
    #     table_name="tensor_kinds",
    #     primary_key_name="kind_name",
    #     conn=conn,
    #     cursor=cursor,
    #     lock=lock,
    # )


"""
`images` stores all images, regardless of type
"""


def create_images_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS images (
                id TEXT PRIMARY KEY,
                origin TEXT,
                image_kind TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY(origin) REFERENCES resource_origins(origin_name),
                FOREIGN KEY(image_kind) REFERENCES image_kinds(kind_name)
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_images_id ON images(id);
            """
        )
        cursor.execute(
            """--sql
            CREATE INDEX IF NOT EXISTS idx_images_origin ON images(origin);
            """
        )
        cursor.execute(
            """--sql
            CREATE INDEX IF NOT EXISTS idx_images_image_kind ON images(image_kind);
            """
        )
        conn.commit()
    finally:
        lock.release()


"""
`images_results` stores additional data specific to `results` images.
"""


def create_images_results_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS images_results (
                images_id TEXT PRIMARY KEY,
                session_id TEXT NOT NULL,
                node_id TEXT NOT NULL,
                FOREIGN KEY(images_id) REFERENCES images(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_images_results_images_id ON images_results(images_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


"""
`images_intermediates` stores additional data specific to `intermediates` images
"""


def create_images_intermediates_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS images_intermediates (
                images_id TEXT PRIMARY KEY,
                session_id TEXT NOT NULL,
                node_id TEXT NOT NULL,
                FOREIGN KEY(images_id) REFERENCES images(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_images_intermediates_images_id ON images_intermediates(images_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


"""
`images_metadata` stores basic metadata for any image type
"""


def create_images_metadata_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS images_metadata (
                images_id TEXT PRIMARY KEY,
                metadata TEXT,
                FOREIGN KEY(images_id) REFERENCES images(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_images_metadata_images_id ON images_metadata(images_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


# `tensors` table: stores references to tensor


def create_tensors_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS tensors (
                id TEXT PRIMARY KEY,
                origin TEXT,
                tensor_kind TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY(origin) REFERENCES resource_origins(origin_name),
                FOREIGN KEY(tensor_kind) REFERENCES tensor_kinds(kind_name)
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_tensors_id ON tensors(id);
            """
        )
        cursor.execute(
            """--sql
            CREATE INDEX IF NOT EXISTS idx_tensors_origin ON tensors(origin);
            """
        )
        cursor.execute(
            """--sql
            CREATE INDEX IF NOT EXISTS idx_tensors_tensor_kind ON tensors(tensor_kind);
            """
        )
        conn.commit()
    finally:
        lock.release()


# `tensors_results` stores additional data specific to `result` tensor


def create_tensors_results_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS tensors_results (
                tensors_id TEXT PRIMARY KEY,
                session_id TEXT NOT NULL,
                node_id TEXT NOT NULL,
                FOREIGN KEY(tensors_id) REFERENCES tensors(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_tensors_results_tensors_id ON tensors_results(tensors_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


# `tensors_intermediates` stores additional data specific to `intermediate` tensor


def create_tensors_intermediates_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS tensors_intermediates (
                tensors_id TEXT PRIMARY KEY,
                session_id TEXT NOT NULL,
                node_id TEXT NOT NULL,
                FOREIGN KEY(tensors_id) REFERENCES tensors(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_tensors_intermediates_tensors_id ON tensors_intermediates(tensors_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


# `tensors_metadata` table: stores generated/transformed metadata for tensor


def create_tensors_metadata_table(conn: sqlite3.Connection, cursor: sqlite3.Cursor, lock: threading.Lock):
    try:
        lock.acquire()

        cursor.execute(
            """--sql
            CREATE TABLE IF NOT EXISTS tensors_metadata (
                tensors_id TEXT PRIMARY KEY,
                metadata TEXT,
                FOREIGN KEY(tensors_id) REFERENCES tensors(id) ON DELETE CASCADE
            );
            """
        )
        cursor.execute(
            """--sql
            CREATE UNIQUE INDEX IF NOT EXISTS idx_tensors_metadata_tensors_id ON tensors_metadata(tensors_id);
            """
        )
        conn.commit()
    finally:
        lock.release()


In [43]:
import os
db_path = '/home/bat/Documents/Code/outputs/test.db'
if (os.path.exists(db_path)):
    os.remove(db_path)

conn = sqlite3.connect(
    db_path, check_same_thread=False
)
cursor = conn.cursor()
lock = threading.Lock()

In [44]:
create_sql_table_from_enum(
    enum=ResourceOrigin,
    table_name="resource_origins",
    primary_key_name="origin_name",
    conn=conn,
    cursor=cursor,
    lock=lock,
)

create_sql_table_from_enum(
    enum=ImageKind,
    table_name="image_kinds",
    primary_key_name="kind_name",
    conn=conn,
    cursor=cursor,
    lock=lock,
)

create_sql_table_from_enum(
    enum=TensorKind,
    table_name="tensor_kinds",
    primary_key_name="kind_name",
    conn=conn,
    cursor=cursor,
    lock=lock,
)

In [45]:
create_images_table(conn, cursor, lock)
create_images_results_table(conn, cursor, lock)
create_images_intermediates_table(conn, cursor, lock)
create_images_metadata_table(conn, cursor, lock)

In [46]:
create_tensors_table(conn, cursor, lock)
create_tensors_results_table(conn, cursor, lock)
create_tensors_intermediates_table(conn, cursor, lock)
create_tensors_metadata_table(conn, cursor, lock)

In [59]:

from pydantic import StrictStr


class GeneratedImageOrLatentsMetadata(BaseModel):
    """Core generation metadata for an image/tensor generated in InvokeAI.

    Generated by traversing the execution graph, collecting the parameters of the nearest ancestors of a given node.

    Full metadata may be accessed by querying for the session in the `graph_executions` table.
    """

    positive_conditioning: Optional[StrictStr] = Field(
        default=None, description="The positive conditioning."
    )
    negative_conditioning: Optional[str] = Field(
        default=None, description="The negative conditioning."
    )
    width: Optional[int] = Field(
        default=None, description="Width of the image/tensor in pixels."
    )
    height: Optional[int] = Field(
        default=None, description="Height of the image/tensor in pixels."
    )
    seed: Optional[int] = Field(
        default=None, description="The seed used for noise generation."
    )
    cfg_scale: Optional[float] = Field(
        default=None, description="The classifier-free guidance scale."
    )
    steps: Optional[int] = Field(
        default=None, description="The number of steps used for inference."
    )
    scheduler: Optional[str] = Field(
        default=None, description="The scheduler used for inference."
    )
    model: Optional[str] = Field(
        default=None, description="The model used for inference."
    )
    strength: Optional[float] = Field(
        default=None,
        description="The strength used for image-to-image/tensor-to-tensor.",
    )
    image: Optional[str] = Field(
        default=None, description="The ID of the initial image."
    )
    tensor: Optional[str] = Field(
        default=None, description="The ID of the initial tensor."
    )
    # Pending model refactor:
    # vae: Optional[str] = Field(default=None,description="The VAE used for decoding.")
    # unet: Optional[str] = Field(default=None,description="The UNet used dor inference.")
    # clip: Optional[str] = Field(default=None,description="The CLIP Encoder used for conditioning.")




In [61]:
GeneratedImageOrLatentsMetadata(positive_conditioning='123')

GeneratedImageOrLatentsMetadata(positive_conditioning='123', negative_conditioning=None, width=None, height=None, seed=None, cfg_scale=None, steps=None, scheduler=None, model=None, strength=None, image=None, tensor=None)